# Read Messages by Topic

This tutorial uses the `TopicMessageReader` to read messages from a robolog by topic. It's one of several reader types available, each designed for different use cases.

In [1]:
import os
from pathlib import Path

current_dir = Path.cwd()
project_root = None

while current_dir != current_dir.parent:
    if (current_dir / '.git').exists():
        project_root = current_dir
        break
    current_dir = current_dir.parent

os.chdir(project_root)

A `TopicMessageReader` can be created by using the factory function `make_topic_message_reader`.

In [2]:
from src.reader import factory

reader = factory.make_topic_message_reader("doc/tutorials/data/ros2")

These are the available topics to read from.

In [3]:
reader.topics

['/fluid_pressure', '/rosout', '/parameter_events', '/events/write_split']

This example reads messages from two topics and casts them into a pandas DataFrame. The resulting table contains four columns:

- `robolog_id`: A unique ID generated for this robolog.
- `timestamp_seconds`: The timestamp of each message in seconds.
- `/rosout` and `/fluid_pressure`: The actual message content for each topic.

The `None` values appear in the topic columns because the message timestamps are not perfectly aligned between the different topics. This will be covered in more detail later.

In [4]:
dataset = reader.read(["/rosout", "/fluid_pressure"])

dataset.to_table().to_pandas()

,robolog_id,timestamp_seconds,/rosout,/fluid_pressure
0,281e260f-0fc9-57e6-8c33-787dc436009f,1.689970e+09,"{'DEBUG': 10, 'INFO': 20, 'WARN': 30, 'ERROR':...",None
1,281e260f-0fc9-57e6-8c33-787dc436009f,1.689970e+09,"{'DEBUG': 10, 'INFO': 20, 'WARN': 30, 'ERROR':...",None
2,281e260f-0fc9-57e6-8c33-787dc436009f,1.689970e+09,"{'DEBUG': 10, 'INFO': 20, 'WARN': 30, 'ERROR':...",None
3,281e260f-0fc9-57e6-8c33-787dc436009f,1.689970e+09,"{'DEBUG': 10, 'INFO': 20, 'WARN': 30, 'ERROR':...",None
4,281e260f-0fc9-57e6-8c33-787dc436009f,1.689970e+09,"{'DEBUG': 10, 'INFO': 20, 'WARN': 30, 'ERROR':...",None
5,281e260f-0fc9-57e6-8c33-787dc436009f,1.689970e+09,"{'DEBUG': 10, 'INFO': 20, 'WARN': 30, 'ERROR':...",None
6,281e260f-0fc9-57e6-8c33-787dc436009f,1.689970e+09,"{'DEBUG': 10, 'INFO': 20, 'WARN': 30, 'ERROR':...",None
7,281e260f-0fc9-57e6-8c33-787dc436009f,1.689970e+09,None,"{'header': {'stamp': {'sec': 0, 'nanosec': 0},..."
8,281e260f-0fc9-57e6-8c33-787dc436009f,1.689970e+09,None,"{'header': {'stamp': {'sec': 0, 'nanosec': 0},..."
9,281e260f-0fc9-57e6-8c33-787dc436009f,1.689970e+09,None,"{'header': {'stamp': {'sec': 0, 'nanosec': 0},..."


The reader also supports filtering messages by a specific time range.

In [5]:
dataset = reader.read(
    ["/rosout", "/fluid_pressure"],
    start_seconds=1689969665.5,
    end_seconds=1689969666.5,
)

dataset.to_table().to_pandas()

,robolog_id,timestamp_seconds,/rosout,/fluid_pressure
0,281e260f-0fc9-57e6-8c33-787dc436009f,1.689970e+09,"{'DEBUG': 10, 'INFO': 20, 'WARN': 30, 'ERROR':...",None
1,281e260f-0fc9-57e6-8c33-787dc436009f,1.689970e+09,None,"{'header': {'stamp': {'sec': 0, 'nanosec': 0},..."


To handle the `None` values caused by misaligned timestamps, set `ffill=True`.

This applies a **forward fill**, which propagates the last valid message forward to fill any gaps until the next message arrives. This technique is also known as "last observation carried forward" (LOCF).

In [6]:
dataset = reader.read(["/rosout", "/fluid_pressure"], ffill=True)

dataset.to_table().to_pandas()

,robolog_id,timestamp_seconds,/rosout,/fluid_pressure
0,281e260f-0fc9-57e6-8c33-787dc436009f,1.689970e+09,"{'DEBUG': 10, 'INFO': 20, 'WARN': 30, 'ERROR':...",None
1,281e260f-0fc9-57e6-8c33-787dc436009f,1.689970e+09,"{'DEBUG': 10, 'INFO': 20, 'WARN': 30, 'ERROR':...",None
2,281e260f-0fc9-57e6-8c33-787dc436009f,1.689970e+09,"{'DEBUG': 10, 'INFO': 20, 'WARN': 30, 'ERROR':...",None
3,281e260f-0fc9-57e6-8c33-787dc436009f,1.689970e+09,"{'DEBUG': 10, 'INFO': 20, 'WARN': 30, 'ERROR':...",None
4,281e260f-0fc9-57e6-8c33-787dc436009f,1.689970e+09,"{'DEBUG': 10, 'INFO': 20, 'WARN': 30, 'ERROR':...",None
5,281e260f-0fc9-57e6-8c33-787dc436009f,1.689970e+09,"{'DEBUG': 10, 'INFO': 20, 'WARN': 30, 'ERROR':...",None
6,281e260f-0fc9-57e6-8c33-787dc436009f,1.689970e+09,"{'DEBUG': 10, 'INFO': 20, 'WARN': 30, 'ERROR':...",None
7,281e260f-0fc9-57e6-8c33-787dc436009f,1.689970e+09,"{'DEBUG': 10, 'INFO': 20, 'WARN': 30, 'ERROR':...","{'header': {'stamp': {'sec': 0, 'nanosec': 0},..."
8,281e260f-0fc9-57e6-8c33-787dc436009f,1.689970e+09,"{'DEBUG': 10, 'INFO': 20, 'WARN': 30, 'ERROR':...","{'header': {'stamp': {'sec': 0, 'nanosec': 0},..."
9,281e260f-0fc9-57e6-8c33-787dc436009f,1.689970e+09,"{'DEBUG': 10, 'INFO': 20, 'WARN': 30, 'ERROR':...","{'header': {'stamp': {'sec': 0, 'nanosec': 0},..."
